## GCF annotation release by NCBI

In [1]:
parameter_file = '../parameters/params.yaml'
import os
import yaml
import sys
from os.path import join
import glob
import pandas as pd

with open(parameter_file,'r') as f:
    doc = yaml.load(f)

#p = dic2obj(**doc)

data_folder = doc['data_folder']
tissues = doc['tissues'].split(',')
sys.path.append(doc['pipeline_path'])
ref_fa = doc['ref_fa']
annotation=doc['annotation']

#tss_file = doc['tss_annotation']

## change to results directory
os.chdir(doc["results"])
tss_file = join("genome/picr/gcf/start_site_mRNA.tsv")
%load_ext autoreload
%autoreload 2


from tss.data import annotation
#from Homer import *
print('Number of tissues: ',len(tissues))
import numpy as np
!pwd


/home/isshamie/software/anaconda2/envs/TSS/lib/python3.7/site-packages/ipykernel_launcher.py:10: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  # Remove the CWD from sys.path while we load stuff.


params {'EMAIL': 'isshamie@ucsd.edu', 'parameters': 'parameters/params.yaml'}
Number of tissues:  13
/data/isshamie/TSS/Analysis/Analysis_03302020/Results


### Create annotation

#### Annotation file:  updated_final_sort.gff3
#### Genome: picr.gff3

In [2]:
outdir = "tss_annotation"
outdir = join(outdir, "picr", "gcf")
print(outdir)

tss_annotation/picr/gcf


In [3]:
if not os.path.exists(outdir):
    os.makedirs(outdir)

In [4]:
peaks_file = "merged/peaks_with_tss_distances_size1.noCDS.tsv"

## Gene

### a.
i. introns excluded  
ii. peak_bins [-1000,+100]

In [5]:
%%time
#peaks_file = 'merged/peaks_with_tss_distances.tsv'
peaks_expression_file = 'merged/samples.merge.peaksexpression'


df = annotation.wrap_create_anno_centric(peaks_file, peaks_expression_file, tss_file,
                             peak_bin=(-1000, 100), anno_col='Nearest gene',
                             tss_df_col='gene',
                             f_save=join(outdir, 'gene_df_01'), allow_intron=False)



100%|██████████| 19774/19774 [16:08<00:00, 20.42it/s] 


Total Number:  21387
Total Number with start sites:  14259
Fraction with start sites:  0.6667134240426428
CPU times: user 12min 33s, sys: 5.64 s, total: 12min 39s
Wall time: 16min 27s


### b.
i. introns excluded  
ii. peak_bins [-1000,+1000]

In [6]:
%%time
#peaks_file = 'merged/peaks_with_tss_distances.tsv'
peaks_expression_file = 'merged/samples.merge.peaksexpression'


df = annotation.wrap_create_anno_centric(peaks_file, peaks_expression_file, tss_file,
                             peak_bin=(-1000, 1000), anno_col='Nearest gene',
                             tss_df_col='gene',
                             f_save=join(outdir, 'gene_df_02'), allow_intron=False)



100%|██████████| 19774/19774 [28:07<00:00, 11.72it/s]  


Total Number:  21387
Total Number with start sites:  15312
Fraction with start sites:  0.7159489409454342
CPU times: user 15min 7s, sys: 6.86 s, total: 15min 14s
Wall time: 28min 21s


## Txn

### a.
i. introns excluded  
ii. peak_bins [-1000,+100]

In [5]:
#peaks_file = 'merged/peaks_with_tss_distances.tsv'
peaks_expression_file = 'merged/samples.merge.peaksexpression'
peaks_df = pd.read_csv(peaks_file,sep='\t',index_col=0,na_filter=False)

tss_df = pd.read_csv(tss_file, sep='\t', index_col=0)
peaks_df = pd.read_csv(peaks_file,sep='\t',index_col=0,na_filter=False)
peak_tissue_matrix = pd.read_csv(peaks_expression_file,sep='\t',index_col=0)
peaks_df.head()

,Chr,Start,End,Strand,Stat,Annotation,Nearest TSS,Distance to TSS,isSameStrand,Nearest gene
ID,,,,,,,,,,
0,NW_020822452.1,16590385,16590386,+,16.227333,promoter-TSS (ID=exon-XM_027399668.1-1;Parent=...,XM_027399668.1,-462.5,False,Clic4
1,NW_020822597.1,23595396,23595397,+,5.740000,Intergenic,XM_027420351.1,-23849.5,True,Baz2b
2,NW_020822680.1,728718,728719,+,7.579579,promoter-TSS (ID=exon-XM_027432478.1-3;Parent=...,XM_027432478.1,746.0,False,Irak1
3,NW_020822602.1,10883205,10883206,+,3.992000,Intergenic,XM_027423054.1,9079.5,True,Gbgt1
4,NW_020822601.1,37427907,37427908,+,129.031738,promoter-TSS (ID=exon-XM_027421990.1-1;Parent=...,XM_027421990.1,-10.5,True,Shf


In [6]:
%%time
#peaks_file = 'merged/peaks_with_tss_distances.tsv'
peaks_expression_file = 'merged/samples.merge.peaksexpression'


df = annotation.wrap_create_anno_centric(peaks_file, peaks_expression_file, tss_file,
                             peak_bin=(-1000, 100), anno_col='Nearest TSS',
                             tss_df_col='transcript_id',
                             f_save=join(outdir, 'txn_df_01'), allow_intron=False)



100%|██████████| 30356/30356 [26:04<00:00, 19.41it/s]  


Total Number:  35679
Total Number with start sites:  18482
Fraction with start sites:  0.5180077916981978
CPU times: user 16min 40s, sys: 5.98 s, total: 16min 46s
Wall time: 26min 16s


### b.
i. introns excluded  
ii. peak_bins [-1000,+1000]

In [7]:
%%time
#peaks_file = 'merged/peaks_with_tss_distances.tsv'
peaks_expression_file = 'merged/samples.merge.peaksexpression'


df = annotation.wrap_create_anno_centric(peaks_file, peaks_expression_file, tss_file,
                             peak_bin=(-1000, 1000), anno_col='Nearest TSS',
                             tss_df_col='transcript_id',
                             f_save=join(outdir, 'txn_df_02'), allow_intron=False)



100%|██████████| 30356/30356 [28:19<00:00, 17.86it/s]  


Total Number:  35679
Total Number with start sites:  20628
Fraction with start sites:  0.5781552173547465
CPU times: user 18min 13s, sys: 8.8 s, total: 18min 22s
Wall time: 28min 34s


In [10]:
f_in = join(outdir,"txn_df_02.p")
f_mat_in = 'merged/samples.merge.peaksexpression'
f_out = join(outdir, 'txn_df_02_matrix')
txn_df_02_matrix = annotation.df_to_TSS_expression(f_in, f_mat_in, f_out=f_out)
txn_df_02_matrix.head()

35679it [07:12, 82.43it/s] 


,Results/f04_peaks/MiscOrgans_START1_and_MiscOrgans_startinput1,Results/f04_peaks/Liver_START2_and_Liver_startinput1,Results/f04_peaks/Brain_GROCap1_and_Brain_GRO1,Results/f04_peaks/Liver_START1_and_Liver_startinput1,Results/f04_peaks/Lung_GROCap1_and_Lung_GRO1,Results/f04_peaks/Brain_START3_and_Brain_startinput1,Results/f04_peaks/Heart_START1_and_Heart_startinput1,Results/f04_peaks/CHO_GROCap2_and_CHO_GRO1,Results/f04_peaks/Heart_START2_and_Heart_startinput1,Results/f04_peaks/Liver_GROCap1_and_Liver_GRO1,...,Results/f04_peaks/MiscOrgans_START2_and_MiscOrgans_startinput1,Results/f04_peaks/Spleen_START1_and_Spleen_startinput1,Results/f04_peaks/Lung_START2_and_Lung_startinput1,Results/f04_peaks/CHO_START1_and_CHO_startinput1,Results/f04_peaks/FemaleReproductive_START1_and_FemaleReproductive_startinput1,Results/f04_peaks/Kidney_START2_and_Kidney_startinput1,Results/f04_peaks/Kidney_GROCap1_and_Kidney_GRO1,Results/f04_peaks/CHO_GROCap1_and_CHO_GRO1,Results/f04_peaks/BMDMwt_GROCap1_and_BMDMwt_GRO1,Results/f04_peaks/Spleen_START2_and_Spleen_startinput1
XM_027386529.1,2.54,0.0,1.59,0.0,59.200001,0.00,0.0,0.0,0.000000,5.06,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,27.900000,0.00,0.00,5.94
XM_027386528.1,4.23,248.5,3.01,0.0,37.099998,4.39,22.1,0.0,130.899994,4.49,...,5.17,32.400002,11864.599609,0.000000,37.799999,116.599998,35.099998,0.00,1.32,7.32
XM_027386531.1,3.38,0.0,1.43,0.0,0.000000,2.10,0.0,0.0,0.000000,0.00,...,0.97,6.270000,0.000000,15.300000,1.730000,0.000000,0.000000,9.97,2.83,0.00
XM_027386533.1,0.00,0.0,0.00,0.0,0.000000,0.00,0.0,0.0,0.000000,0.00,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00
XM_027386534.1,5.92,12.5,12.90,0.0,8.690000,21.40,7.7,0.0,12.000000,6.18,...,2.42,20.000000,57.200001,32.700001,22.000000,16.200001,3.670000,0.00,19.40,5.94
